# Nighttime Lights Trends in Gaza and West Bank

Analyzing conflict dynamics through the lens of [NASA's Black Marble Nighttime Lights](https://blackmarble.gsfc.nasa.gov) dataset opens a unique window into the often-hidden facets of global unrest. In a world marked by diverse forms of conflict, from armed confrontations to civil unrest, the dataset offers an unconventional yet powerful tool for understanding the ripple effects of these conflicts on human settlements and infrastructure. By tracking nighttime light variations and disruptions, we can unearth vital insights into population displacement, economic destabilization, and the societal impacts of conflict. This analysis explores the potential of the Black Marble Nighttime Lights dataset to not only detect areas affected by conflict but also to quantify the extent of its influence on human lives and livelihoods, providing a valuable perspective on the multifaceted consequences of conflict worldwide.

In [1]:
import math
import os
from datetime import datetime
from math import pi

import colorcet as cc
import dask.dataframe as dd
import folium
import geopandas
import numpy as np
import pandas as pd
from blackmarble.bm_extract import bm_extract
from blackmarble.bm_raster import bm_raster
from bokeh.layouts import column as BOKEH_COLUMN
from bokeh.layouts import gridplot
from bokeh.models import (
    BasicTicker,
    ColumnDataSource,
    Div,
    HoverTool,
    Legend,
    PrintfTickFormatter,
    Range1d,
    Span,
    TabPanel,
    Tabs,
    Text,
    Title,
)
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import linear_cmap

In [2]:
group = lambda flat, size: [flat[i : i + size] for i in range(0, len(flat), size)]

## Data

### Define Region of Interest

Define region of interest for where NASA Black Marble will be downloaded.

In [3]:
PSE = geopandas.read_file("../../data/boundaries/gadm41_PSE_shp/gadm41_PSE_2.shp")

ROI = PSE
ROI.explore()

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Region of Interest. Country borders or names do not necessarily reflect the World Bank Group's official position. This map is for illustrative purposes and does not imply the expression of any opinion on the part of the World Bank, conceming the legal status of any country or territory or concerning the delimitation of frontiers or boundaries. 
```

### Black Marble 

[NASA's Black Marble](https://blackmarble.gsfc.nasa.gov) VIIRS (Visible Infrared Imaging Radiometer Suite) Nighttime Lights dataset represents a remarkable advancement in our ability to monitor and understand nocturnal light emissions on a global scale. By utilizing cutting-edge satellite technology and image processing techniques, the Black Marble VIIRS dataset offers a comprehensive and high-resolution view of the Earth's nighttime illumination patterns. 

In [4]:
dates = (
    pd.date_range("2023-11-01", "2023-11-27", freq="D").strftime("%Y-%m-%d").tolist()
)

df = bm_extract(
    roi_sf=ROI,
    product_id="VNP46A2",
    date=dates,
    bearer=os.environ.get("BLACKMARBLE_TOKEN"),
    output_location_type="file",
    file_dir="data",
    file_prefix="pse2_",
    aggregation_fun=["count", "mean", "min", "max", "median", "sum"],
    quiet=True,
)

In [5]:
df = dd.read_csv(
    "data/pse2_VNP46A2*.csv",
    parse_dates=["date"],
).compute()

The latest update date is:

In [6]:
df["date"].max().strftime("%d %B %Y (Week %W)")

'14 November 2023 (Week 46)'

```{important}
The [VNP46A2 Daily Moonlight-adjusted Nighttime Lights (NTL) Product](https://blackmarble.gsfc.nasa.gov/VNP46A2.html) is [available daily](datahttps://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/VNP46A2/#data-availability). However, due data quality, cloud cover or other factors, the data may not be available always at a specific location.
```

## Methodology

Creating a time series of weekly radiance using NASA's Black Marble data involves several steps, including data acquisition, pre-processing, zonal statistics calculation, and time series generation. Below is a general methodology for this process.

###  Time Series Generation

Organize the zonal statistics results in a tabular format, where each columnn corresponds to a specific zone, and rows represent the daily radiance values. Next, we aggregate the data on a weekly basis, computing the desired statistical metric (e.g., mean radiance) for each zone for each week. Finally, we will visualize the time series data to observe trends, patterns, and anomalies over time.

#### Weekly

In this step, we compute a weekly aggregation of the zonal statistics by for each second-level administrative division and for each week. In this case, we `W-SUN` and `mean` as aggregate function.

In [7]:
# https://papermill.readthedocs.io/en/latest/usage-parameterize.html
VAR = "ntl_mean"

In [8]:
PSE_2 = (
    df.pivot_table(values=[VAR], index="date", columns=["NAME_2"])
    .resample("W-SUN", label="right")
    .mean()
)
PSE_1 = (
    df.pivot_table(values=[VAR], index="date", columns=["NAME_1"])
    .resample("W-SUN", label="right")
    .mean()
)

PSE_2

ntl_mean                                                \
NAME_2       Bethlehem Deir Al-Balah        Gaza Gaza ash Shamaliyah   
date                                                                   
2022-01-02  156.367935     80.876536  135.911132           78.008197   
2022-01-09   39.567083    133.175823  203.156980          118.514480   
2022-01-16   91.582545    120.802306  154.040451          108.742094   
2022-01-23  118.228905    134.793033  197.818275          131.700803   
2022-01-30   40.004512     97.599245  151.641740           94.552836   
...                ...           ...         ...                 ...   
2023-10-22   72.135592     15.493020   31.566008           22.924328   
2023-10-29   46.712910     19.406778   61.449688           30.709801   
2023-11-05   92.355671     21.139765   71.703914           39.983443   
2023-11-12   75.788061     15.325895   27.945485           23.456580   
2023-11-19   81.835625     12.103516   13.703561           18.975309   

                                                                       \
NAME_2          Hebron       Jenin    Jericho   Jerusalem  Khan Yunis   
date                                                                    
2022-01-02   55.146905   49.865671  55.028910  268.081904  121.089891   
2022-01-09   67.958752   75.255281  55.975712  218.802689  135.848382   
2022-01-16   49.600785   64.682987  44.844365  295.262626  129.915630   
2022-01-23   69.660681   72.395834  43.670591  229.732588  123.835625   
2022-01-30   45.156670   46.628070  44.336758  186.807820  108.125086   
...                ...         ...        ...         ...         ...   
2023-10-22   95.210494   86.556170  63.660436  265.335301   18.401297   
2023-10-29   79.991052   96.933238  66.465023  147.185100   27.215036   
2023-11-05  101.384094  105.661603  63.510271  258.907989   29.838535   
2023-11-12  109.787658  103.118763  61.243796  211.688490   19.052143   
2023-11-19   81.962455   86.249665  65.614536  275.244381   14.583559   

                                                                      \
NAME_2          Nablus    Qalqilya       Rafah Ramallah and Al-Bireh   
date                                                                   
2022-01-02   71.561816  111.464012         NaN             99.026577   
2022-01-09   69.845524  172.819059  225.478376             97.225112   
2022-01-16   70.264066  151.807916  218.354316             94.762849   
2022-01-23   80.544432  181.125428  201.245460             88.265481   
2022-01-30   49.063802  126.010498  165.917218             65.618373   
...                ...         ...         ...                   ...   
2023-10-22   91.947717  180.900032   48.671099            109.332663   
2023-10-29   99.002919  201.183602   58.474931            135.834073   
2023-11-05  108.353341  200.006728   61.242155            121.457682   
2023-11-12   95.151817  158.595085   82.154826            103.427940   
2023-11-19   67.463855  181.158940   41.475409            131.310136   

                                               
NAME_2          Salfit      Tubas     Tulkarm  
date                                           
2022-01-02  143.262311  36.491211   56.065521  
2022-01-09  149.484119  24.412828  113.316800  
2022-01-16  121.237199  19.542756  114.116851  
2022-01-23  139.010203  28.030261  118.935038  
2022-01-30  105.499612  22.234509   84.132638  
...                ...        ...         ...  
2023-10-22  156.763529  39.359721  127.670345  
2023-10-29  166.433177  49.659227  135.130657  
2023-11-05  177.311697  45.650336  144.443296  
2023-11-12  189.560895  41.245775  127.090813  
2023-11-19  145.839094  42.837350  135.849657  

[99 rows x 16 columns]

#### Monthly

In this step, we compute a monthy aggregation of the zonal statistics by for each second-level administrative division and for each month. Additionaly, we add the [VNP46A3](https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/VNP46A3/#data-availability) monthly composite, when available.

In [9]:
PSE_2_MS = (
    df.pivot_table(index="date", columns=["NAME_2"], values=[VAR], aggfunc="mean")
    .resample("MS", label="left")
    .mean()
)
PSE_2_MS.to_csv("../../data/ntl/PSE_2-VNP46A2-M-zs.csv")

In [10]:
VNP46A3 = (
    (
        dd.read_csv(
            "data/pse2_VNP46A3*.csv",
            parse_dates=["date"],
        )
        .compute()
        .pivot_table(index="date", columns=["NAME_2"], values=[VAR], aggfunc="mean")
    )
    .resample("MS", label="left")
    .mean()
)

In [11]:
p = figure(
    title="Palestine: Monthly Nighttime Lights (2022-2023)",
    width=800,
    height=600,
    x_axis_label="Date",
    x_axis_type="datetime",
    y_axis_label=r"Radiance [nW $$cm^{-2}$$ $$sr^{-1}$$]",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
)
p.add_layout(
    Title(
        text=f"Monthly NTL Radiance Average (VNP46A2 and VNP46A3) for each second-level administrative division",
        text_font_size="12pt",
        text_font_style="italic",
    ),
    "above",
)
p.add_layout(
    Title(
        text=f"Data Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
        text_font_size="10pt",
        text_font_style="italic",
    ),
    "below",
)
p.add_layout(Legend(), "right")

p.add_tools(
    HoverTool(
        tooltips=[
            ("Month", "@x{%B %Y}"),
            ("Radiance", "@y{0.00}"),
        ],
        formatters={"@x": "datetime"},
    )
)
renderers = []
for column, color in zip(PSE_2_MS.columns, cc.b_glasbey_category10):
    try:
        r = p.line(
            PSE_2_MS.index,
            PSE_2_MS[column],
            legend_label=column[1],
            line_color=color,
            line_width=2,
        )
        r.visible = False
        renderers.append(r)
    except:
        pass
renderers[-1].visible = True

for column, color in zip(VNP46A3.columns, cc.b_cyclic_grey_15_85_c0):
    r = p.line(
        VNP46A3.index,
        VNP46A3[column],
        legend_label=column[1],
        line_color=color,
        line_width=2,
    )
    r.visible = False
    renderers.append(r)
renderers[-1].visible = True

p.legend.location = "bottom_left"
p.legend.click_policy = "hide"
p.title.text_font_size = "16pt"

output_notebook()
show(p)

Loading BokehJS ...

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Monthly NTL radiance average derived from the VNP46A2 daily composite and, in grey, derived from the VNP46A3 monthly composite.
```

## Findings

### Percent Change in NTL Radiance 

#### Benchmark Comparison

In this exploratory analysis, we conducted analysis of NTL radiance trends, comparing the observed average radiance levels to a benchmark established in the year 2022 for each second-level administrative division.

In [12]:
PERCENTCHANGE_2 = 100 * (
    PSE_2 / PSE_2[(PSE_2.index >= "2022-01-01") & (PSE_2.index < "2023-01-01")].mean()
    - 1
)
PERCENTCHANGE_1 = 100 * (
    PSE_1 / PSE_1[(PSE_1.index >= "2022-01-01") & (PSE_2.index < "2023-01-01")].mean()
    - 1
)
pd.set_option("display.max_rows", None)
PERCENTCHANGE_2[PERCENTCHANGE_2.index >= "2023-10-01"].style.map(
    lambda x: "background-color: #DF4661" if x < -50 else "background-color: white"
)

In [13]:
def create_plot(data):
    p = figure(
        title="Palestine: Percent Change in Nighttime Lights Radiance Average",
        width=800,
        height=600,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Percent Change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    # p.y_range = Range1d(-100, 50, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi / 4
    p.add_layout(
        Title(
            text=f"Weekly percent change (2022 baseline) in NTL radiance for each second-level administrative division",
            text_font_size="12pt",
            text_font_style="italic",
        ),
        "above",
    )
    p.add_layout(
        Title(
            text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
            text_font_size="10pt",
            text_font_style="italic",
        ),
        "below",
    )
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="gray",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips=[
                ("Date", "@x{%F} (@x{%W of %Y})"),
                ("Percent Change", "@y{0.0}%"),
            ],
            formatters={"@x": "datetime"},
        )
    )
    renderers = []
    for column, color in zip(data.columns, cc.b_glasbey_category10):
        r = p.line(
            data.index,
            data[column],
            legend_label=str(column[1]),
            line_color=color,
            line_width=2,
        )
        r.visible = False
        renderers.append(r)
    renderers[2].visible = True

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "16pt"
    # p.sizing_mode = "scale_both"
    return p

In [14]:
PERCENTCHANGE_2.to_csv("../../data/ntl/PSE_2-VNP46A2-WSUN-percentchange.csv")

In [15]:
show(create_plot(PERCENTCHANGE_2))

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Weekly percent change (compared to 2022 benchmark) in NTL radiance (VNP46A2) for each second-level administrative division. 
```

Additionally, we visualize below the NTL radiance average on first-level administrative division. 

In [16]:
def create_plot_individual(data, colors):
    p = figure(
        title=data.columns[0][1],
        width=800,
        height=600,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Percent Change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    p.y_range = Range1d(-100, 40, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi / 4
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="red",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips=[
                ("Date", "@x{%F} (@x{%W of %Y})"),
                ("Percent Change", "@y{0.0}% (2022 baseline)"),
            ],
            formatters={"@x": "datetime"},
        )
    )
    p.add_layout(
        Title(
            text=f"Weekly percent change (2022 baseline) in NTL radiance",
            text_font_size="12pt",
            text_font_style="italic",
        ),
        "above",
    )
    renderers = []
    for column, color in zip(data.columns, colors):
        r = p.line(
            data.index,
            data[column],
            line_color=color,
            line_width=2,
        )
        renderers.append(r)

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "24pt"
    # p.sizing_mode = "scale_both"

    return p

In [17]:
plots = list()

for column, color in zip(PERCENTCHANGE_1.columns, cc.bmy):
    p = create_plot_individual(PERCENTCHANGE_1[column].to_frame(), [color])
    plots.append(p)
    show(p)

# for column, color in zip(PERCENTCHANGE_2.columns, cc.b_glasbey_category10):
#     p = create_plot_individual(PERCENTCHANGE_2[column].to_frame(), [color])
#     plots.append(p)

# p = gridplot(group(plots, 1))
# p.sizing_mode = "scale_height"

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Weekly percent change (compared to 2022 benchmark) in NTL radiance (NASA Black Marble VNP46A2) for each first-level administrative division. 
```

#### Week over Week Comparison

In this exploratory analysis, we conducted analysis of NTL radiance trends, comparing the observed average radiance levels week over week (WOW) for each second-level administrative division.

In [18]:
WOW_2 = 100 * (PSE_2.pct_change(1, axis="rows", fill_method=None))

In [19]:
WOW_2.to_csv("../../data/ntl/PSE_2-VNP46A2-WOW-percentchange.csv")

In [20]:
p = figure(
    title="Palestine: Percent Change in Nighttime Lights Radiance",
    width=800,
    height=600,
    x_axis_label="Date",
    x_axis_type="datetime",
    y_axis_label="NTL Percent Change (%)",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
)
p.xaxis.major_label_orientation = math.pi / 4
p.add_layout(
    Title(
        text=f"Weekly percent change week over week in NTL radiance for each second-level administrative division",
        text_font_size="12pt",
        text_font_style="italic",
    ),
    "above",
)
p.add_layout(
    Title(
        text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
        text_font_size="10pt",
        text_font_style="italic",
    ),
    "below",
)
p.add_layout(Legend(), "right")
p.renderers.extend(
    [
        Span(
            location=datetime(2023, 10, 7),
            dimension="height",
            line_color="gray",
            line_width=1.5,
            line_dash=(4, 4),
        ),
    ]
)
p.add_tools(
    HoverTool(
        tooltips=[
            ("Week", "@x{%W} (@x{%F})"),
            ("Percent Change", "@y{0.00}% (week over week)"),
        ],
        formatters={"@x": "datetime"},
    )
)
renderers = []
for column, color in zip(WOW_2.columns, cc.b_glasbey_category10):
    r = p.line(
        WOW_2.index,
        WOW_2[column],
        legend_label=str(column[1]),
        line_color=color,
        line_width=2,
    )
    r.visible = False
    renderers.append(r)

renderers[2].visible = True

p.legend.location = "bottom_left"
p.legend.click_policy = "hide"
p.title.text_font_size = "16pt"
# p.sizing_mode = "scale_both"

show(p)

### Point-in-Time Comparison

#### Daily

```{figure} ./figures/pse_ntl_VNP46A1_2023-01-01.png
---
height: 400px
---
Nighttime lights on January 1, 2023. Source: NASA Black Marble (VNP46A1).
```

```{figure} ./figures/pse_ntl_VNP46A1_2023-10-21.png
---
height: 400px
---
Nighttime lights on October 21, 2023. Source: NASA Black Marble (VNP46A1).
```

#### Weekly
We visualize below weekly snapshots of the percent change (compared to 2022) in NTL radiance average for each second-level administrative division. 

In [21]:
m = PSE.merge(
    PERCENTCHANGE_2.iloc[1]
    .to_frame("percent change in radiance average (%)")
    .reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="cividis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{PERCENTCHANGE_2.iloc[0].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

In [22]:
m = PSE.merge(
    PERCENTCHANGE_2.loc["2023-10-08"]
    .to_frame("percent change in radiance average (%)")
    .reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="cividis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{PERCENTCHANGE_2.loc["2023-10-08"].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

In [23]:
m = PSE.merge(
    PERCENTCHANGE_2.iloc[-1]
    .to_frame("percent change in radiance average (%)")
    .reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="cividis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{PERCENTCHANGE_2.iloc[-1].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

## Limitations 

Using nighttime lights to estimate macroeconomic indicators during conflict may be a valuable approach, but it comes with several assumptions and limitations. Here's a list of some of the key assumptions and limitations:

```{caution}
**Assumptions:**

- **Luminosity Reflects Economic Activity:** The approach assumes that the level of nighttime lights is a reliable proxy for economic activity. It presupposes that areas with brighter lights correspond to higher economic productivity.

- **Baseline Data Availability:** It assumes the availability of baseline nighttime lights data before the onset of the conflict. The accuracy of the estimates depends on the quality and relevance of this baseline data.

- **Spatial Distribution:** The method assumes that nighttime lights are evenly distributed within a given geographic area and that changes in luminosity accurately reflect changes in economic activity across all locations.
    

**Limitations:**

- **Confounding Factors and Data Interpretation:** The approach may require subjective interpretation, as it may not distinguish between reduced lighting due to conflict and reduced lighting due to other factors. Changes in nighttime lights can be influenced by factors other than economic activity, such as energy conservation measures, urban development, or seasonal variations.

- **Generalization:** The approach might lead to overgeneralization, as a reduction in nighttime lights can be associated with various economic outcomes, from minor disruptions to severe economic downturns.

- **Alternative Explanations:** Changes in nighttime lights can result from factors other than conflict, such as urban development, changes in economic activities, or natural disasters. Therefore, it may not always be clear whether a decline in nighttime lights is solely due to conflict.

- **Geopolitical Factors:** The dataset may be subject to geopolitical biases, with some areas having less comprehensive coverage due to political reasons.

- **Data Lag:** There can be a significant time lag between the occurrence of a conflict event and its reflection in the nighttime lights dataset. This lag may limit the dataset's utility for real-time conflict monitoring.

- **Resolution and Urban Bias:** The dataset's spatial resolution may not be fine enough to capture small villages or isolated conflict events. It may also have an urban bias, making it less suitable for analyzing rural or remote conflicts.
```

To address these assumptions and limitations, it is crucial to complement nighttime lights data analysis with other sources of information and adopt a cautious and context-aware approach when interpreting the findings.

## References

{cite:empty}`ROMAN2018113`

```{bibliography}
:filter: docname in docnames
:style: plain
```